Lambda School Data Science

*Unit 2, Sprint 1, Module 3*

---

# Ridge Regression

## Assignment

We're going back to our other **New York City** real estate dataset. Instead of predicting apartment rents, you'll predict property sales prices.

But not just for condos in Tribeca...

- [ ] Use a subset of the data where `BUILDING_CLASS_CATEGORY` == `'01 ONE FAMILY DWELLINGS'` and the sale price was more than 100 thousand and less than 2 million.
- [ ] Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.
- [ ] Do one-hot encoding of categorical features.
- [ ] Do feature selection with `SelectKBest`.
- [ ] Fit a ridge regression model with multiple features. Use the `normalize=True` parameter (or do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html) beforehand — use the scaler's `fit_transform` method with the train set, and the scaler's `transform` method with the test set)
- [ ] Get mean absolute error for the test set.
- [ ] As always, commit your notebook to your fork of the GitHub repo.

The [NYC Department of Finance](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page) has a glossary of property sales terms and NYC Building Class Code Descriptions. The data comes from the [NYC OpenData](https://data.cityofnewyork.us/browse?q=NYC%20calendar%20sales) portal.


## Stretch Goals

Don't worry, you aren't expected to do all these stretch goals! These are just ideas to consider and choose from.

- [ ] Add your own stretch goal(s) !
- [ ] Instead of `Ridge`, try `LinearRegression`. Depending on how many features you select, your errors will probably blow up! 💥
- [ ] Instead of `Ridge`, try [`RidgeCV`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html).
- [ ] Learn more about feature selection:
    - ["Permutation importance"](https://www.kaggle.com/dansbecker/permutation-importance)
    - [scikit-learn's User Guide for Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html)
    - [mlxtend](http://rasbt.github.io/mlxtend/) library
    - scikit-learn-contrib libraries: [boruta_py](https://github.com/scikit-learn-contrib/boruta_py) & [stability-selection](https://github.com/scikit-learn-contrib/stability-selection)
    - [_Feature Engineering and Selection_](http://www.feat.engineering/) by Kuhn & Johnson.
- [ ] Try [statsmodels](https://www.statsmodels.org/stable/index.html) if you’re interested in more inferential statistical approach to linear regression and feature selection, looking at p values and 95% confidence intervals for the coefficients.
- [ ] Read [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf), Chapters 1-3, for more math & theory, but in an accessible, readable way.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

In [43]:
%%capture
import numpy as np
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [258]:
!pip install pandas_profiling
import pandas as pd
import pandas_profiling

# Read New York City property sales data
df = pd.read_csv(DATA_PATH+'condos/NYC_Citywide_Rolling_Calendar_Sales.csv')

# Change column names: replace spaces with underscores
df.columns = [col.replace(' ', '_') for col in df]

# SALE_PRICE was read as strings.
# Remove symbols, convert to integer
df['SALE_PRICE'] = (
    df['SALE_PRICE']
    .str.replace('$','')
    .str.replace('-','')
    .str.replace(',','')
    .astype(int)
)
#df['LAND_SQUARE_FEET']=df['LAND_SQAURE_FEET'].str.replace('########','')

In [259]:
# BOROUGH is a numeric column, but arguably should be a categorical feature,
# so convert it from a number to a string
df['BOROUGH'] = df['BOROUGH'].astype(str)

In [260]:
df

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,...,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
0,1,CHELSEA,13 CONDOS - ELEVATOR APARTMENTS,2,716,1246,NaN,R4,"447 WEST 18TH STREET, PH12A",PH12A,...,1.0,0.0,1.0,"10,733",1979.0,2007.0,2,R4,0,01/01/2019
1,1,FASHION,21 OFFICE BUILDINGS,4,812,68,NaN,O5,144 WEST 37TH STREET,NaN,...,0.0,6.0,6.0,"2,962",15435.0,1920.0,4,O5,0,01/01/2019
2,1,FASHION,21 OFFICE BUILDINGS,4,839,69,NaN,O5,40 WEST 38TH STREET,NaN,...,0.0,7.0,7.0,"2,074",11332.0,1930.0,4,O5,0,01/01/2019
3,1,GREENWICH VILLAGE-WEST,13 CONDOS - ELEVATOR APARTMENTS,2,592,1041,NaN,R4,"1 SHERIDAN SQUARE, 8C",8C,...,1.0,0.0,1.0,0,500.0,0.0,2,R4,0,01/01/2019
4,1,UPPER EAST SIDE (59-79),15 CONDOS - 2-10 UNIT RESIDENTIAL,2C,1379,1402,NaN,R1,"20 EAST 65TH STREET, B",B,...,1.0,0.0,1.0,0,6406.0,0.0,2,R1,0,01/01/2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23035,4,ST. ALBANS,01 ONE FAMILY DWELLINGS,1,10965,276,NaN,A5,111-17 FRANCIS LEWIS BLVD,NaN,...,1.0,0.0,1.0,"1,800",1224.0,1945.0,1,A5,510000,04/30/2019
23036,4,SUNNYSIDE,09 COOPS - WALKUP APARTMENTS,2,169,29,NaN,C6,"45-14 43RD STREET, 3C",NaN,...,0.0,0.0,0.0,0,0.0,1929.0,2,C6,355000,04/30/2019
23037,4,SUNNYSIDE,10 COOPS - ELEVATOR APARTMENTS,2,131,4,NaN,D4,"50-05 43RD AVENUE, 3M",NaN,...,0.0,0.0,0.0,0,0.0,1932.0,2,D4,375000,04/30/2019
23038,4,WOODHAVEN,02 TWO FAMILY DWELLINGS,1,8932,18,NaN,S2,91-10 JAMAICA AVE,NaN,...,2.0,1.0,3.0,"2,078",2200.0,1931.0,1,S2,1100000,04/30/2019


In [261]:
# Reduce cardinality for NEIGHBORHOOD feature

# Get a list of the top 10 neighborhoods
top10 = df['NEIGHBORHOOD'].value_counts()[:10].index

# At locations where the neighborhood is NOT in the top 10, 
# replace the neighborhood with 'OTHER'
df.loc[~df['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'

In [262]:
df['NEIGHBORHOOD'].value_counts()

OTHER                      19117
FLUSHING-NORTH               685
UPPER EAST SIDE (59-79)      465
UPPER EAST SIDE (79-96)      410
BEDFORD STUYVESANT           408
BOROUGH PARK                 376
UPPER WEST SIDE (59-79)      350
GRAMERCY                     323
ASTORIA                      312
FOREST HILLS                 298
UPPER WEST SIDE (79-96)      296
Name: NEIGHBORHOOD, dtype: int64

In [210]:
df['LAND_SQUARE_FEET']

0        10,733
1         2,962
2         2,074
3             0
4             0
          ...  
23035     1,800
23036         0
23037         0
23038     2,078
23039    15,151
Name: LAND_SQUARE_FEET, Length: 23040, dtype: object

In [214]:
# df['LAND_SQUARE_FEET']=df['LAND_SQUARE_FEET'].str.replace(',','')
# df['LAND_SQUARE_FEET']=pd.to_numeric(df['LAND_SQUARE_FEET'])
# #df[pd.to_numeric(df['LAND_SQUARE_FEET'], errors='coerce').isnull()]
# #df.applymap(np.isreal)

In [263]:
# Use a subset of the data where BUILDING_CLASS_CATEGORY == '01 ONE FAMILY DWELLINGS' 
#and the sale price was more than 100 thousand and less than 2 million.

condition = (df['BUILDING_CLASS_CATEGORY']=='01 ONE FAMILY DWELLINGS') & (df['SALE_PRICE']>100000) & (df['SALE_PRICE']<2000000)

In [264]:
df['SALE_DATE']=pd.to_datetime(df['SALE_DATE'])
print(df['SALE_DATE'].dtype)

subset=df[condition]

datetime64[ns]


Now let's divide them into test and train data. 

In [265]:
print(df.shape)
df.describe().T

(23040, 21)


,count,mean,std,min,25%,50%,75%,max
BLOCK,23040.0,4.459116e+03,3.713270e+03,1.0,1341.0,3546.0,6673.75,16350.0
LOT,23040.0,3.539485e+02,6.280253e+02,1.0,22.0,49.0,375.00,9057.0
EASE-MENT,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ZIP_CODE,23039.0,1.079119e+04,1.103414e+03,0.0,10306.0,11211.0,11360.00,11697.0
RESIDENTIAL_UNITS,23039.0,1.827423e+00,1.009043e+01,0.0,0.0,1.0,2.00,750.0
COMMERCIAL_UNITS,23039.0,2.865142e-01,5.778224e+00,-148.0,0.0,0.0,0.00,570.0
TOTAL_UNITS,23039.0,2.320891e+00,1.207055e+01,0.0,1.0,1.0,2.00,755.0
GROSS_SQUARE_FEET,23039.0,3.508555e+03,2.218610e+04,0.0,494.0,1356.0,2280.00,1303935.0
YEAR_BUILT,23005.0,1.819672e+03,4.883766e+02,0.0,1920.0,1940.0,1965.00,2019.0
TAX_CLASS_AT_TIME_OF_SALE,23040.0,1.624479e+00,8.038978e-01,1.0,1.0,1.0,2.00,4.0


In [266]:

subset.describe(include='object').T

,count,unique,top,freq
BOROUGH,3151,5,4,1580
NEIGHBORHOOD,3151,6,OTHER,2990
BUILDING_CLASS_CATEGORY,3151,1,01 ONE FAMILY DWELLINGS,3151
TAX_CLASS_AT_PRESENT,3151,2,1,3111
BUILDING_CLASS_AT_PRESENT,3151,13,A1,1185
ADDRESS,3151,3135,294 FREEBORN STREET,2
APARTMENT_NUMBER,1,1,RP.,1
LAND_SQUARE_FEET,3151,1035,"4,000",289
BUILDING_CLASS_AT_TIME_OF_SALE,3151,11,A1,1186


In [267]:
train = subset[subset['SALE_DATE'].dt.month <4]
test = subset[subset['SALE_DATE'].dt.month==4]

In [268]:
#Now let's prepare our features and taret vectors, and do onehot encoding on them. 

target='SALE_PRICE'
to_drop=['APARTMENT_NUMBER', 'ADDRESS', 'BUILDING_CLASS_CATEGORY', 'EASE-MENT', 'SALE_DATE']
features = train.columns.drop([target]+to_drop)
features


Index(['BOROUGH', 'NEIGHBORHOOD', 'TAX_CLASS_AT_PRESENT', 'BLOCK', 'LOT',
       'BUILDING_CLASS_AT_PRESENT', 'ZIP_CODE', 'RESIDENTIAL_UNITS',
       'COMMERCIAL_UNITS', 'TOTAL_UNITS', 'LAND_SQUARE_FEET',
       'GROSS_SQUARE_FEET', 'YEAR_BUILT', 'TAX_CLASS_AT_TIME_OF_SALE',
       'BUILDING_CLASS_AT_TIME_OF_SALE'],
      dtype='object')

In [269]:

X_train = train[features]
X_test = test[features]
y_train = train[target] 
y_test = test[target]


In [270]:
X_train

,BOROUGH,NEIGHBORHOOD,TAX_CLASS_AT_PRESENT,BLOCK,LOT,BUILDING_CLASS_AT_PRESENT,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE
44,3,OTHER,1,5495,801,A9,11230.0,1.0,0.0,1.0,"6,800",1325.0,1930.0,1,A9
61,4,OTHER,1,7918,72,A1,11427.0,1.0,0.0,1.0,"4,000",2001.0,1940.0,1,A1
78,2,OTHER,1,4210,19,A1,10461.0,1.0,0.0,1.0,"3,500",2043.0,1925.0,1,A1
108,3,OTHER,1,5212,69,A1,11226.0,1.0,0.0,1.0,"4,000",2680.0,1899.0,1,A1
111,3,OTHER,1,7930,121,A5,11203.0,1.0,0.0,1.0,"1,710",1872.0,1940.0,1,A5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18129,5,OTHER,1,4081,44,A2,10306.0,1.0,0.0,1.0,"2,400",921.0,1950.0,1,A2
18130,5,OTHER,1,2373,201,A5,10314.0,1.0,0.0,1.0,"2,450",2128.0,1980.0,1,A5
18132,5,OTHER,1,1132,42,A1,10302.0,1.0,0.0,1.0,"4,361",1807.0,2018.0,1,A1
18134,5,OTHER,1,3395,37,A2,10305.0,1.0,0.0,1.0,"6,000",621.0,1930.0,1,A2


In [271]:
!pip install category_encoders


In [272]:
import category_encoders as ce
encoder = ce.OneHotEncoder(use_cat_names=True)
X_train = encoder.fit_transform(X_train)
X_test = encoder.transform(X_test)

In [273]:
X_test

,BOROUGH_3,BOROUGH_4,BOROUGH_2,BOROUGH_5,BOROUGH_1,NEIGHBORHOOD_OTHER,NEIGHBORHOOD_FLUSHING-NORTH,NEIGHBORHOOD_BEDFORD STUYVESANT,NEIGHBORHOOD_FOREST HILLS,NEIGHBORHOOD_BOROUGH PARK,...,BUILDING_CLASS_AT_TIME_OF_SALE_A1,BUILDING_CLASS_AT_TIME_OF_SALE_A5,BUILDING_CLASS_AT_TIME_OF_SALE_A0,BUILDING_CLASS_AT_TIME_OF_SALE_A2,BUILDING_CLASS_AT_TIME_OF_SALE_A3,BUILDING_CLASS_AT_TIME_OF_SALE_S1,BUILDING_CLASS_AT_TIME_OF_SALE_A4,BUILDING_CLASS_AT_TIME_OF_SALE_A6,BUILDING_CLASS_AT_TIME_OF_SALE_A8,BUILDING_CLASS_AT_TIME_OF_SALE_S0
18235,0,0,1,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
18239,0,0,1,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
18244,1,0,0,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
18280,1,0,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
18285,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23029,0,1,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
23031,0,1,0,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
23032,0,1,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
23033,0,1,0,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


### Do feature selection with `SelectKBest`.

In [274]:
from sklearn.feature_selection import SelectKBest, f_regression

selector = SelectKBest(score_func=f_regression, k=30)

X_train_selected = selector.fit_transform(X_train, y_train)
X_test_selected = selector.transform(X_test)
print(X_train_selected.shape, X_test_selected.shape)


(2507, 30) (644, 30)


/Applications/anaconda3/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:299: RuntimeWarning:

divide by zero encountered in true_divide

/Applications/anaconda3/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:304: RuntimeWarning:

invalid value encountered in true_divide

/Applications/anaconda3/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning:

invalid value encountered in greater

/Applications/anaconda3/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning:

invalid value encountered in less

/Applications/anaconda3/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:1892: RuntimeWarning:

invalid value encountered in less_equal



In [275]:
#Which features were selected? 

selected_mask=selector.get_support()
all_names=X_train.columns
selected_names=all_names[selected_mask]
unselected_names = all_names[~selected_mask]

print ('Features selected:')
for x in selected_names:
    print(x)
    
    
print ('\n\nFeatures not selected:')
for x in unselected_names:
    print(x)

Features selected:
BOROUGH_3
BOROUGH_4
BOROUGH_2
BOROUGH_5
NEIGHBORHOOD_OTHER
NEIGHBORHOOD_FLUSHING-NORTH
NEIGHBORHOOD_FOREST HILLS
NEIGHBORHOOD_BOROUGH PARK
TAX_CLASS_AT_PRESENT_1
BLOCK
BUILDING_CLASS_AT_PRESENT_A5
BUILDING_CLASS_AT_PRESENT_A3
BUILDING_CLASS_AT_PRESENT_S1
BUILDING_CLASS_AT_PRESENT_A6
BUILDING_CLASS_AT_PRESENT_A8
BUILDING_CLASS_AT_PRESENT_S0
ZIP_CODE
RESIDENTIAL_UNITS
COMMERCIAL_UNITS
TOTAL_UNITS
LAND_SQUARE_FEET_5,000
LAND_SQUARE_FEET_0
LAND_SQUARE_FEET_4,290
GROSS_SQUARE_FEET
BUILDING_CLASS_AT_TIME_OF_SALE_A5
BUILDING_CLASS_AT_TIME_OF_SALE_A3
BUILDING_CLASS_AT_TIME_OF_SALE_S1
BUILDING_CLASS_AT_TIME_OF_SALE_A6
BUILDING_CLASS_AT_TIME_OF_SALE_A8
BUILDING_CLASS_AT_TIME_OF_SALE_S0


Features not selected:
BOROUGH_1
NEIGHBORHOOD_BEDFORD STUYVESANT
NEIGHBORHOOD_ASTORIA
TAX_CLASS_AT_PRESENT_1D
LOT
BUILDING_CLASS_AT_PRESENT_A9
BUILDING_CLASS_AT_PRESENT_A1
BUILDING_CLASS_AT_PRESENT_A0
BUILDING_CLASS_AT_PRESENT_A2
BUILDING_CLASS_AT_PRESENT_A4
BUILDING_CLASS_AT_PRESENT_B2
BUILDI

Try k from 1 to 30 to decide what features should be selected

In [276]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error 

result=[]
for k in range(1,len(X_train.columns)+1):
    print (f'{k} features')
    
    selector=SelectKBest(score_func=f_regression, k=k)
    X_train_selected = selector.fit_transform(X_train, y_train)
    X_test_selected = selector.transform(X_test)
    
    model=LinearRegression()
    model.fit(X_train_selected, y_train)
    y_pred=model.predict(X_test_selected)
    mae=mean_absolute_error(y_test,y_pred)
    print(f'Test Mean Absolute Error: ${mae:,.0f}\n')
    result.append([k,mae.round()])

1 features
Test Mean Absolute Error: $183,641

2 features
Test Mean Absolute Error: $182,569

3 features
Test Mean Absolute Error: $182,569

4 features
Test Mean Absolute Error: $173,706

5 features
Test Mean Absolute Error: $174,556

6 features
Test Mean Absolute Error: $172,843

7 features
Test Mean Absolute Error: $173,412

8 features
Test Mean Absolute Error: $173,241

9 features


/Applications/anaconda3/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:299: RuntimeWarning:

divide by zero encountered in true_divide

/Applications/anaconda3/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:304: RuntimeWarning:

invalid value encountered in true_divide

/Applications/anaconda3/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning:

invalid value encountered in greater

/Applications/anaconda3/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning:

invalid value encountered in less

/Applications/anaconda3/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:1892: RuntimeWarning:

invalid value encountered in less_equal

/Applications/anaconda3/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:299: RuntimeWarning:

divide by zero encountered in true_divide

/Applications/anaconda3/lib/python3.7/site-packages/

Test Mean Absolute Error: $168,668

10 features
Test Mean Absolute Error: $169,452

11 features
Test Mean Absolute Error: $169,006

12 features
Test Mean Absolute Error: $161,221

13 features
Test Mean Absolute Error: $162,578

14 features
Test Mean Absolute Error: $161,733

15 features
Test Mean Absolute Error: $161,735

16 features
Test Mean Absolute Error: $161,640

17 features
Test Mean Absolute Error: $161,449

18 features


/Applications/anaconda3/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:299: RuntimeWarning:

divide by zero encountered in true_divide

/Applications/anaconda3/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:304: RuntimeWarning:

invalid value encountered in true_divide

/Applications/anaconda3/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning:

invalid value encountered in greater

/Applications/anaconda3/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning:

invalid value encountered in less

/Applications/anaconda3/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:1892: RuntimeWarning:

invalid value encountered in less_equal

/Applications/anaconda3/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:299: RuntimeWarning:

divide by zero encountered in true_divide

/Applications/anaconda3/lib/python3.7/site-packages/

Test Mean Absolute Error: $161,449

19 features
Test Mean Absolute Error: $161,205

20 features
Test Mean Absolute Error: $161,205

21 features
Test Mean Absolute Error: $159,212

22 features
Test Mean Absolute Error: $158,538

23 features
Test Mean Absolute Error: $158,605

24 features
Test Mean Absolute Error: $158,605

25 features
Test Mean Absolute Error: $158,605

26 features
Test Mean Absolute Error: $158,605

27 features


...

/Applications/anaconda3/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:299: RuntimeWarning:

divide by zero encountered in true_divide

/Applications/anaconda3/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:304: RuntimeWarning:

invalid value encountered in true_divide

/Applications/anaconda3/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning:

invalid value encountered in greater

/Applications/anaconda3/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning:

invalid value encountered in less

/Applications/anaconda3/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:1892: RuntimeWarning:

invalid value encountered in less_equal

/Applications/anaconda3/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:299: RuntimeWarning:

divide by zero encountered in true_divide

/Applications/anaconda3/lib/python3.7/site-packages/

Test Mean Absolute Error: $1,147,640,085,709

932 features
Test Mean Absolute Error: $152,366,073,166

933 features


/Applications/anaconda3/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:299: RuntimeWarning:

divide by zero encountered in true_divide

/Applications/anaconda3/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:304: RuntimeWarning:

invalid value encountered in true_divide

/Applications/anaconda3/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning:

invalid value encountered in greater

/Applications/anaconda3/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning:

invalid value encountered in less

/Applications/anaconda3/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:1892: RuntimeWarning:

invalid value encountered in less_equal

/Applications/anaconda3/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:299: RuntimeWarning:

divide by zero encountered in true_divide



Test Mean Absolute Error: $1,082,785,519,259



/Applications/anaconda3/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:304: RuntimeWarning:

invalid value encountered in true_divide

/Applications/anaconda3/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning:

invalid value encountered in greater

/Applications/anaconda3/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning:

invalid value encountered in less

/Applications/anaconda3/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:1892: RuntimeWarning:

invalid value encountered in less_equal



In [280]:
result

[[1, 183641.0],
 [2, 182569.0],
 [3, 182569.0],
 [4, 173706.0],
 [5, 174556.0],
 [6, 172843.0],
 [7, 173412.0],
 [8, 173241.0],
 [9, 168668.0],
 [10, 169452.0],
 [11, 169006.0],
 [12, 161221.0],
 [13, 162578.0],
 [14, 161733.0],
 [15, 161735.0],
 [16, 161640.0],
 [17, 161449.0],
 [18, 161449.0],
 [19, 161205.0],
 [20, 161205.0],
 [21, 159212.0],
 [22, 158538.0],
 [23, 158605.0],
 [24, 158605.0],
 [25, 158605.0],
 [26, 158605.0],
 [27, 158340.0],
 [28, 158223.0],
 [29, 158157.0],
 [30, 158115.0],
 [31, 158085.0],
 [32, 158847.0],
 [33, 158902.0],
 [34, 158909.0],
 [35, 158854.0],
 [36, 158857.0],
 [37, 159368.0],
 [38, 160160.0],
 [39, 160265.0],
 [40, 160350.0],
 [41, 160350.0],
 [42, 160504.0],
 [43, 160559.0],
 [44, 160502.0],
 [45, 160614.0],
 [46, 160011.0],
 [47, 160150.0],
 [48, 91404302217.0],
 [49, 76052653316.0],
 [50, 160161.0],
 [51, 52959354932.0],
 [52, 70373012702.0],
 [53, 16954594624.0],
 [54, 169214004838.0],
 [55, 159679.0],
 [56, 40111094671.0],
 [57, 160147.0],
 [58

In [278]:
import plotly.express as px 
resulting=pd.DataFrame(result, columns=['x','y'])
px.scatter(resulting, x='x', y='y', title="Mean Absolute Error for different k values")
#result

- [ ] Fit a ridge regression model with multiple features. Use the `normalize=True` parameter (or do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html) beforehand — use the scaler's `fit_transform` method with the train set, and the scaler's `transform` method with the test set)

In [285]:
from sklearn.linear_model import Ridge

for alpha in [0.001, 0.01, 0.1, 1,10.0,100.0,1000.0]:
    #Fir Ridge Regression model 
    print(f'Ridge Regression, with alpha={alpha}')
    model=Ridge(alpha=alpha, normalize=True)
    model.fit(X_train, y_train)
    y_pred=model.predict(X_test)
    
    #Get test MAE
    mae=mean_absolute_error(y_test, y_pred)
    print(f'MAE: ${mae:,.0f}')
    


Ridge Regression, with alpha=0.001
MAE: $167,261
Ridge Regression, with alpha=0.01
MAE: $166,851
Ridge Regression, with alpha=0.1
MAE: $165,469
Ridge Regression, with alpha=1
MAE: $175,191
Ridge Regression, with alpha=10.0
MAE: $199,813
Ridge Regression, with alpha=100.0
MAE: $209,951
Ridge Regression, with alpha=1000.0
MAE: $211,397


In [289]:
#Check RidgeCV

alpha=[0.001, 0.01, 0.1, 1,10.0,100.0,1000.0]

from sklearn.linear_model import RidgeCV
ridge=RidgeCV(alphas=alpha, normalize=True)
ridge.fit(X_train, y_train)
#y_pred=ridge.predict(X_test)
ridge.alpha_


0.1